In [1]:

import os
from dotenv import load_dotenv
import warnings
warnings.filterwarnings("ignore")


# LangChain 최신 버전 임포트
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.chat_models import ChatOllama
from langchain_community.vectorstores import Chroma 
import chromadb
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser, PydanticOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.documents import Document

load_dotenv()
if not os.environ.get('OPENAI_API_KEY'):
    raise ValueError('.env 확인하세요. key가 없습니다')
if not os.environ.get('LANGSMITH_API_KEY'):
    raise ValueError('LANGSMITH_API_KEY 없음. env 확인해주세요')

'''
LangSmith 연결(env 셋팅 돼있어야 합니다!)
# pip install -U langchain langsmith (한번만 실행)
'''

os.environ["LANGSMITH_TRACING_V2"] = "true" #기본값 false
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_PROJECT"]="pet_rag_multiquery" #프로젝트 이름
print("LangSmith 연결 완료")


USER_AGENT environment variable not set, consider setting it to identify your requests.


LangSmith 연결 완료


In [33]:
from langchain_community.embeddings import HuggingFaceEmbeddings

# 임베딩 모델 로드
embeddings = HuggingFaceEmbeddings(
        model_name="BAAI/bge-m3",
        model_kwargs={'device': 'cpu'},
        encode_kwargs={'normalize_embeddings': True}
    )

#벡터스토어 로드
vectorstore = Chroma(
persist_directory=r"./ChromaDB_bge_m3", #DB 저장 경로
collection_name="pet_health_qa_system_bge_m3",
embedding_function=embeddings)
print("벡터스토어가 성공적으로 로드되었습니다!")

벡터스토어가 성공적으로 로드되었습니다!


In [34]:
#컬렉션 확인
client = chromadb.PersistentClient(path=r"./ChromaDB_bge_m3")
collections = client.list_collections()
print("사용 가능한 컬렉션:", [c.name for c in collections])


# db내 문서 개수
print("총 문서 개수:", vectorstore._collection.count())

사용 가능한 컬렉션: ['pet_health_qa_system_bge_m3']
총 문서 개수: 54786


진료과 필터링도 추가해보고 싶네..

In [35]:
prompt = ChatPromptTemplate.from_messages([
    ("system","반려동물 건강 전문가로서, 사용자의 질문을 3가지 관점에서 재구성한 질문을 제시하세요. \n"
    "각 질문은 /로 구분하세요. 각 질문 앞에 아무것도 붙이지 마세요."),
    ("user","사용자의 질문: {question}"),
])

llm = ChatOpenAI(model="gpt-4o", temperature=0)

multiquery_chain = prompt|llm|StrOutputParser()

query = "우리 강아지가 자주 긁는데 무슨 병일까요?"

querys = multiquery_chain.invoke({"question":query})
print("재구성된 질문들:", querys)

재구성된 질문들: 강아지가 자주 긁는 원인이 될 수 있는 일반적인 피부 질환은 무엇인가요? / 강아지가 자주 긁는 행동이 알레르기와 관련이 있을 수 있나요? / 강아지가 자주 긁는 증상을 완화하기 위해 집에서 할 수 있는 관리 방법은 무엇인가요?


Multi Query Retrieval
    → n개 쿼리 생성
    → n개의 검색 결과 수집
    → 문서 합치기
    → 중복 제거
    → 점수 기반으로 재정렬
    → 최종 top-k 문서를 context로 사용

In [39]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":1})

#포맷팅 함수
def format_docs(docs):
    '''문서를 문자열로 포멧팅'''
    return '\n\n---\n\n'.join([ doc.page_content for doc in docs ])

for q in querys.split('/'):
    print("개별 질문:", q.strip())

    multiquery_docs = []
    print("---각 질문 관련 문서 검색---")
    docs = retriever.invoke(q.strip())
    for d in docs:
        print(d.page_content)
    
    #문서 합치기
    multiquery_docs.append(docs)




개별 질문: 강아지가 자주 긁는 원인이 될 수 있는 일반적인 피부 질환은 무엇인가요?
---각 질문 관련 문서 검색---
Q: 저희 강아지가 자주 긁은 경향을 보이고 있는 것으로 데, 이는 피부 상태가 좋지 않음을 나타내는 것인지 궁금합니다.

A: 관찰하신 내용을 잘 확인하였습니다. 관찰하신 증상으로 보아, 피부에 발적이 있는 것으로 것으로 예상됩니다. 이는 염증에 의해 소양감이 발생하고 있을 가능성이 있습니다. 따라서, 병원에 방문하셔서 강아지의 상태를 정밀하게 확인한 후 적절한 치료가 필요할 수 있을 것으로 생각됩니다. 기본 안전하게 관리는 무리 없이 유지해하는 게 좋습니다 격일로 확인해 주세요 경과 사진을 주 1~2회 정도 남겨 주세요. 매주 가능하면 경과를 보며 강도나 빈도를 조정해 주세요 악화되면 중단하고 진료를 받아 주세요 자극이 있으면 강도를 낮춰 주세요. 매일 경과를 보며 강도나 빈도를 조정해 주세요 악화되면 중단하고 진료를 받아 주세요.
개별 질문: 강아지가 자주 긁는 행동이 알레르기와 관련이 있을 수 있나요?
---각 질문 관련 문서 검색---
A: 몸을 자주 긁고 특정 부위를 집중적으로 긁는 행동은 알레르기 반응의 흔한 증상입니다. 음식 알레르기나 환경 알레르기(꽃가루, 집먼지진드기 등)가 원인일 수 있으며, 현재 급여하시는 인섹트 밀웜 사료나 화식에 대한 반응일 수도 있습니다. 또한, 잦은 피부 자극으로 인해 이차적인 세균 또는 효모 감염이 발생하여 증상을 악화시킬 수도 있습니다.
겨드랑이 부위를 만졌을 때 불쾌감을 보이는 것은 해당 부위의 피부염이나 림프절 비대 등을 의심해 볼 수 있습니다. 과거 유리너리 결석 병력이나 깨갱거리는 증상 등도 현재 상태와 관련이 있을 수 있으므로 수의사에게 자세히 말씀해 주시는 것이 중요합니다.
정확한 원인을 파악하고 효과적인 치료 계획을 세우기 위해서는 동물병원에 방문하여 종합적인 피부 검사를 받아보시는 것을 권해드립니다. 수의사는 피부 상태를 꼼꼼히 확인하고, 필요에 따라 피부 스크래핑, 세포 검사,

In [40]:
print(f"문서를 합칭 결과(= context): {multiquery_docs}")

문서를 합칭 결과(= context): [[Document(metadata={'source_path': '.\\3차 프로젝트\\data\\qa\\TL_질의응답데이터_피부과', 'source_type': 'qa_data', 'chunk_index': 1, 'disease': '기타', 'department': '피부과', 'chunk_method': 'qa_data', 'total_chunks': 3, 'answer': '강아지의 관리에 대해 많은 걱정을 하고 계신 것 같습니다, 불편해하면 강도는 낮춰 주세요. 일반적으로 강아지가 귓병이 나 피부병으로 인해 가려움증을 느껴 심하게 긁게 되면 상처가 발생할 수 있습니다. 이를 예방하기 위한 방법으로 넥칼라, 즉 깔대기를 착용하게 되는 경우가 많습니다. 하지만 평소에 잘 착용하지 않은 강아지들은 이로 인해 큰 불편함을 느끼고 스트레스를 받을 수 있습니다. 만약 보호자께서 집에서 강아지가 긁은 것을 막을 수 있다면 깔대기를 잠시 풀어주셔도 괜찮습니다. 그러나 그러한 상황이 어렵다면 깔대기를 계속 착용하도록 하는 것이 좋습니다. 보다 자세한 사항에 대해서는 보호자님께서 방문하시는 동물병원에 문의하시기를 권장합니다. 지속 가능하게 가정 내 관리는 가능하면 부담 없이 이어가 주세요, 증상 강도 변화를 간단히 기록해 주시고, 중요한 변화만 기록해 주세요. 관리 계획은 점차적으로 꾸준히 시행해 주세요, 가려움이 심한 시간대는 자극을 줄여 주세요. 재발 예방 관리는 과하지 않게 기록을 남기며 진행하시고, 피부를 긁지 않도록 관리해 주세요. 가정 내 관리는 무리가 가지 않게 무리 없이 이어가 주세요, 경과 사진을 날짜와 함께 남겨 주세요, 불편해하면 강도는 낮춰 주세요. 관리 습관은 서서히 규칙적으로 진행해 주시고, 필요하면 계획을 조정해 주세요. 피부 관리는 일정하게 꾸준히 시행하시고, 오염 부위는 미지근한 물로 가볍게 세척해 주세요. 생활 관리는 일정하게 변화에 맞춰 조절하시고, 오염 부위는 미지근한 물로 가볍게 세척해 주세요

In [41]:
context = multiquery_docs
prompt = ChatPromptTemplate.from_messages([
        ("system", """
당신은 반려견 질병·증상에 대해 수의학 정보를 제공하는 AI 어시스턴트입니다. 
당신의 답변은 반드시 제공된 문맥(Context)만을 기반으로 해야 합니다.
문맥에 없는 정보는 절대로 추측하거나 생성하지 마세요.

[사용 가능한 정보 유형]
- medical_data: 수의학 서적 또는 논문
- qa_data: 보호자-수의사 상담 기록 (생애주기 / 과 / 질병 태그 포함)

[할루시네이션 방지 규칙]
1. 문맥에 없는 정보는 사용하지 마세요.
2. 관련 정보가 없다면 “해당 질문과 관련된 문서를 찾지 못했습니다. ”라고 답변하세요.
3. 여러 문서 제공시, 실제로 답변에 사용한 문서만 출처 명시하세요.
4. **질문에 합당한 답변만 제공하세요. 거짓 정보나 불필요한 정보는 제외하세요.**

[응답 규칙]
- 보호자가 작성한 반려견 상태를 2~3문장으로 요약한다.
- 문맥에서 확인된 가능한 원인을 구체적으로 설명한다. 
  (문맥에 없다면 “문서에 해당 정보가 없습니다”라고 쓴다)
- 집에서 가능한 안전한 관리 방법 2~3개 제안한다. 
  (문맥에 없다면 제안하지 않는다)
- 언제 병원에 가야 하는지, 어떤 증상이 응급인지 문서 기반으로 설명한다.
- 마지막 줄에 반드시 출처를 명시한다:
  • 서적 출처: 책 제목 / 저자 / 출판사
  • QA 출처: 생애주기 / 과 / 질병

[전체 톤]
- 공손한 존댓말
- 보호자를 안심시키되, 필요한 부분은 명확하게 안내하는 수의사 상담 톤
         

[출력 형식]
-상태 요약:
-가능한 원인:
-집에서 관리 방법:
-병원 방문 시기:
-출처(참고한 모든 문서)

"""),
("human",
"""
문맥: {context}

사용자 질문: {question}
""")
    ])


In [ ]:
rag_chain = prompt|llm|StrOutputParser()
answer = rag_chain.invoke({"context":})